In [2]:
from sklearn.feature_extraction import DictVectorizer
import pymysql
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from langdetect import detect
import csv
import MySQLdb
import pandas as pd


In [9]:
def execute_sql(conn, query):
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    cur.execute(query)
    return cur

conn = MySQLdb.connect(host='127.0.0.1', user='root', passwd="", db='vroniplag', charset='utf8')

In [4]:
def get_query_monolingual_table(word_ratio_min, word_ratio_max, len_min, bow_diff_min=1):
    query = "CREATE TABLE monolingual AS SELECT annotation_identifier, url, "
    query += "plagiat_sent, source_sent, fake_source_sent, lang_plagiat, lang_source, "
    query += "bow_diff, nb_words_ratio "
    query += "from annotationAugmented "
    query += "WHERE category='Verschleierung' and lang_plagiat=lang_source and "
    query += "nb_words_ratio>" + str(word_ratio_min) + " and nb_words_ratio<" + str(word_ratio_max) + " and "
    query += "length(plagiat_sent)>" + str(len_min) + " and length(source_sent)>" + str(len_min) + " and "
    query += "bow_diff>" + str(bow_diff_min)
    query += ";"
    print(query)
    return query

def get_query_monolingual_lang_table(tablename, lang):
    query = "CREATE TABLE " + tablename + " as SELECT annotation_identifier, url, plagiat_sent, source_sent, "
    query += "bow_diff, nb_words_ratio "
    query += "from monolingual where lang_plagiat='" + lang + "' and lang_source='" + lang + "';"
    print(query)
    return query

**Get data**

In [28]:
def get_data_from_monolingual_table(lang, keys, isParaphrase, excludeInnerIdenticals=False):
    query = "SELECT * from monolingual" + lang.upper() + " WHERE isParaphrase=" + str(isParaphrase) + ";"
    cur = execute_sql(conn, query)
    rows = []
    counter = 0
    for row in cur:
        #if row['annotation_identifier'] != "Aaf/Fragment 009 01_7":
        #    continue
            
        new_row = []
        for key in keys:
            val = row[key]
            #print(type(val))
            if type(val)==unicode:
                #print(val)
                #val = val.decode('latin-1').encode("utf-8")
                val = val.encode("utf-8")
                #print(val)
                #print("")
                
            new_row.append(val)
        if excludeInnerIdenticals:
            plag = row['plagiat_sent'].lower()
            src = row['source_sent'].lower()
            if plag in src or src in plag:
                counter += 1
                continue    
        rows.append(new_row)
    
    print("# skipped (inner identicals): " + str(counter))
    return rows

def split_data(percent_test, data):
    split = int(percent_test * len(data))
    test_data = data[:split+1]
    train_data = data[split+1:]
    return train_data, test_data


#### Include negative paraphrase pairs

In [21]:
def addValToQuery(vals, query):
    for idx, key in enumerate(vals):
        if idx==len(vals)-1:
            query += str(key) + ") "
        else:
            query += str(key) + ", "
    return query
            

In [19]:
def addNegatives(conn, tablename, fakeSrcDict):

    cur = execute_sql(conn, "SELECT * from " + tablename + " WHERE isParaphrase=1;")
    for row in cur:
        annotation_id = row['annotation_identifier']
        fake_src = fakeSrcDict[annotation_id]
        if not fake_src: continue
        row['annotation_identifier'] = annotation_id + "_f"
        row['source_sent'] = fake_src
        row['isParaphrase'] = 0
        
        query = "INSERT INTO " + tablename + " ("
        query = addValToQuery(row.keys(), query)
        query += " VALUES ("
        values = []
        for item in row.values():
            if type(item) == str:
                item = item.replace("'","\\'")
            values.append("'"+str(item)+"'")
                
        query = addValToQuery(values, query)
        query += ";"
        try:
            cur2 = conn.cursor()
            cur2.execute(query)
            cur2.close()
        except Exception as e:
            ## Fehler passieren durch Abostrophe im Text ... 
            print(row['url'])
            if row['url'] == 'http://de.vroniplag.wikia.com/wiki/Ww/Fragment_016_01' or row['url'] == 'http://de.vroniplag.wikia.com/wiki/Yb/Fragment_186_14':
                print(query)

    conn.commit()

**Fake Sources**

In [16]:
# Get mapping from annotation_identifier to fake source sent
def getFakeSources(lang):
    query = "SELECT * from monolingual WHERE lang_plagiat='"+lang+"' and lang_source='"+lang+"';"
    cur_annotation = execute_sql(conn, query)
    ids_to_fake_sources = {}
    for row in cur_annotation:
        key = row['annotation_identifier']
        ids_to_fake_sources[key] = row['fake_source_sent']

    return ids_to_fake_sources
    cur_annotation.close()

**Generate CSV-Files in UTF-8**

In [24]:
get_data_from_monolingual_table('es', keys, 1)

# skipped (inner identicals): 0


[]

In [26]:
def write_csv_files(lang, keys):
    data_isPP = get_data_from_monolingual_table(lang, keys, 1)
    print("nb paraphrases: " + str(len(data_isPP)))
    data_noPP = get_data_from_monolingual_table(lang, keys, 0)
    print("nb fake-paraphrases: " + str(len(data_noPP)))

    train_data_isPP, test_data_isPP = split_data(0.2, data_isPP)
    train_data_noPP, test_data_noPP = split_data(0.2, data_noPP)

    train_data = pd.DataFrame(train_data_isPP + train_data_noPP,columns=keys)
    test_data = pd.DataFrame(test_data_isPP + test_data_noPP, columns=keys)

    print("train shape: " + str(train_data.shape))
    print("test shape: " + str(test_data.shape))
    train_data.to_csv(lang+'_train.csv', encoding='utf-8')
    test_data.to_csv(lang+'_test.csv', encoding='utf-8')

#### Enrich the *annotation*-table with the columns **category**, **lang_source** and **lang_plagiat** from the *fragment*-table


In [9]:
query = "CREATE TABLE annotationAugmented AS SELECT * from annotation"
cur = execute_sql(conn, query)
conn.commit()
cur.close()

In [10]:
query = "ALTER TABLE annotationAugmented ADD COLUMN lang_source VARCHAR(80), ADD COLUMN lang_plagiat VARCHAR(80), "
query += "ADD COLUMN category VARCHAR(80);"
print(query)
cur = execute_sql(conn, query)
conn.commit()
cur.close()

ALTER TABLE annotationAugmented ADD COLUMN lang_source VARCHAR(80), ADD COLUMN lang_plagiat VARCHAR(80), ADD COLUMN category VARCHAR(80);


In [11]:
# Insert data
cur_fragment = execute_sql(conn, "SELECT * from fragment")
count=0
for row in cur_fragment:
    url = row['url']
    lang_original = row['lang_source']
    lang_plagiat = row['lang_plagiat']
    category = row['category']
    query = "UPDATE annotationAugmented SET lang_source='" + lang_original + "', lang_plagiat='" + lang_plagiat;
    query += "', category='" + category;
    query += "' WHERE url='" + url + "';"
    execute_sql(conn, query)
    if(count % 1000 == 0):
        print("{0} processed".format(count))
    count+=1
    
conn.commit()
cur_fragment.close()

0 processed
1000 processed
2000 processed
3000 processed
4000 processed
5000 processed
6000 processed
7000 processed
8000 processed
9000 processed
10000 processed
11000 processed
12000 processed
13000 processed
14000 processed
15000 processed


#### Create Monolingual database from *annotationAugmented*-table

In [11]:
query = get_query_monolingual_table(0.5, 1.5, 70, 6)
cur = execute_sql(conn, query)
conn.commit()
cur.close()

CREATE TABLE monolingual AS SELECT annotation_identifier, url, plagiat_sent, source_sent, fake_source_sent, lang_plagiat, lang_source, bow_diff, nb_words_ratio from annotationAugmented WHERE category='Verschleierung' and lang_plagiat=lang_source and nb_words_ratio>0.5 and nb_words_ratio<1.5 and length(plagiat_sent)>70 and length(source_sent)>70 and bow_diff>6;


In [7]:
keys = ['plagiat_sent', 'source_sent', 'isParaphrase', 'url']

**MonolingualEN**-table

In [13]:
query = get_query_monolingual_lang_table("monolingualEN", "en")
cur = execute_sql(conn, query)
conn.commit()
cur.close()

CREATE TABLE monolingualEN as SELECT annotation_identifier, url, plagiat_sent, source_sent, bow_diff, nb_words_ratio from monolingual where lang_plagiat='en' and lang_source='en';


In [14]:
query = "ALTER TABLE monolingualEN ADD COLUMN isParaphrase TINYINT(1) DEFAULT 1"
cur = execute_sql(conn, query)
conn.commit()
cur.close()

In [22]:
ids_to_fake_sources = getFakeSources("en")
addNegatives(conn, "monolingualEN", ids_to_fake_sources)

In [31]:
# write train and test csv files
write_csv_files("en", keys)

# skipped (inner identicals): 0
nb paraphrases: 1938
# skipped (inner identicals): 0
nb fake-paraphrases: 1580
train shape: (2813, 4)
test shape: (705, 4)


**MonolingualDE**-table

In [23]:
query = get_query_monolingual_lang_table("monolingualDE", "de")
cur = execute_sql(conn, query)
conn.commit()
cur.close()

CREATE TABLE monolingualDE as SELECT annotation_identifier, url, plagiat_sent, source_sent, bow_diff, nb_words_ratio from monolingual where lang_plagiat='de' and lang_source='de';


In [24]:
query = "ALTER TABLE monolingualDE ADD COLUMN isParaphrase TINYINT(1) DEFAULT 1"
cur = execute_sql(conn, query)
conn.commit()
cur.close()

In [25]:
ids_to_fake_sources = getFakeSources("de")
addNegatives(conn, "monolingualDE", ids_to_fake_sources)

In [30]:
# write train and test csv files
write_csv_files("de", keys)

# skipped (inner identicals): 0
nb paraphrases: 12932
# skipped (inner identicals): 0
nb fake-paraphrases: 10935
train shape: (19092, 4)
test shape: (4775, 4)


**MonolingualES**-table

In [26]:
query = get_query_monolingual_lang_table("monolingualES", "es")
cur = execute_sql(conn, query)
conn.commit()
cur.close()

CREATE TABLE monolingualES as SELECT annotation_identifier, url, plagiat_sent, source_sent, bow_diff, nb_words_ratio from monolingual where lang_plagiat='es' and lang_source='es';


In [27]:
query = "ALTER TABLE monolingualES ADD COLUMN isParaphrase TINYINT(1) DEFAULT 1"
cur = execute_sql(conn, query)
conn.commit()
cur.close()

In [28]:
###Einbauen, dass jedes annotation_identifier nur einmal eingefügt werden darf!! D.h. jedes negative Beispiel darf
### nur einmal eingebaut werden!!
# MonolingualES
ids_to_fake_sources = getFakeSources("es")
addNegatives(conn, "monolingualES", ids_to_fake_sources)

In [29]:
# write train and test csv files
write_csv_files("es", keys)

# skipped (inner identicals): 0
nb paraphrases: 7
# skipped (inner identicals): 0
nb fake-paraphrases: 5
train shape: (8, 4)
test shape: (4, 4)


In [26]:
conn.close()